In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('preprocessed.csv')
df

,Path,Text
0,pack\AIBR\2006-Aymaras-peruanos-y-chilenos-en-...,catalogada y caricaturizada bajo el rótulo de ...
1,pack\AIBR\2006-Cambios-de-género-y-discriminac...,agradecimientos la autora desea agradecer a la...
2,pack\AIBR\2006-Consecuencias-personales-en-la-...,has carried out the biggest employment adjustm...
3,pack\AIBR\2006-Construcción-de-modelos-de-géne...,tampoco queremos que se entienda identidad de ...
4,pack\AIBR\2006-Contribuciones-feministas-a-pro...,is irrelevant for others con el que teresa del...
...,...,...
1920,pack\Revista-Española-de-Antropología-American...,"cómo citar: burguete cal y mayor, araceli. 202..."
1921,pack\Revista-Española-de-Antropología-American...,project exposed some critical questionings. in...
1922,pack\Revista-Española-de-Antropología-American...,"1. introducción el interés por los quipus, que..."
1923,pack\Revista-Española-de-Antropología-American...,1. introducción ¿cómo se produce la diferencia...


In [3]:
import torch
import spacy
from transformers import pipeline
from dotenv import load_dotenv
import os

load_dotenv()

nlp = spacy.load("es_core_news_sm")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(torch.cuda.current_device()))

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
df.iloc[2]['Path']

'pack\\AIBR\\2006-Consecuencias-personales-en-la-ruptura-de-la-vida-laboral-El-caso-de-Telefónica.pdf'

### QA basico

In [ ]:
from llama_index.core import VectorStoreIndex, Document
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login
import os
from dotenv import load_dotenv

model_id = "meta-llama/Llama-3.2-1B-Instruct"

# Cargar variables de entorno
load_dotenv()

# Obtener el token desde .env
token = os.getenv("HUGGINGFACE_TOKEN")
login(token)

# Configurar el modelo LLM de Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Configurar LlamaIndex para usar Hugging Face como LLM
llm = HuggingFaceLLM(model=model, tokenizer=tokenizer)

# Configurar un modelo de embeddings local o alternativo
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Cargar el texto (desde un DataFrame en este caso)
text = df.iloc[2]['Text']

# Crear documento
document = Document(text=text)

# Crear índice CON el modelo de embeddings específico
index = VectorStoreIndex.from_documents(
    [document], 
    embed_model=embed_model
)

# Crear motor de consulta con el modelo de Hugging Face
query_engine = index.as_query_engine(llm=llm)

# Hacer una pregunta basada en el texto
# respuesta = query_engine.query("¿Cuál es la idea principal del texto?")
# respuesta = query_engine.query("¿que es la ley de estatuto de los trabajadores?")
respuesta = query_engine.query("¿de que tipos pueden ser los flujos de mercado de trabajo?") 
print(respuesta)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 La ley de estatuto de los trabajadores es una norma legal que establece las condiciones de trabajo, los derechos y las obligaciones de los trabajadores en un país. En España, por ejemplo, la ley de estatuto de los trabajadores es el Decreto 39/2003, de 28 de junio, que regula el estatuto de los trabajadores en el ámbito laboral español. El Decreto 39/2003 establece las normas básicas para el trabajo, incluyendo la regulación de los derechos y las obligaciones de los trabajadores, así como la protección de los trabajadores en la empresa.  La ley de estatuto de los trabajadores tiene como objetivo principal el proteger los derechos y las condiciones de trabajo de los trabajadores, garantizando su dignidad y bienestar. En España, el estatuto de los trabajadores es un derecho fundamental, y su cumplimiento es obligatorio para las empresas que emplean a los trabajadores. La ley de estatuto de los trabajadores también establece la posibilidad de la negociación colectiva, lo que permite a lo

In [9]:
respuesta = query_engine.query("¿de que tipos pueden ser los flujos de mercado de trabajo?") 
print(respuesta)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 Los flujos de mercado de trabajo pueden ser de dos tipos:

1. Flujos de mercado de trabajo tradicionales: son los flujos de mercado de trabajo que siguen las formas tradicionales de empleo, donde el empleador busca a un empleado que tenga una antigüedad en la empresa y que esté dispuesto a trabajar en un horario estable.
2. Flujos de mercado de trabajo innovadores: son los flujos de mercado de trabajo que siguen las formas innovadoras de empleo, donde el empleador busca a un empleado que tenga habilidades y competencias que no se encuentran en la empresa, y que estén dispuestos a trabajar en un horario flexible o en un entorno de trabajo más dinámico.


Langchain

In [ ]:
import pickle

In [10]:
embeddings_np = np.load("embeddings-chunks-512-50-nomic.bin.npy")
embeddings_np.shape

(73111, 768)

In [ ]:
with open("chunks-512-50-nomic.pkl", "rb") as f:
    chunks, chunks_ids = pickle.load(f)